# Comparison of All Three Methods

This notebook compares results from all three NER extraction methods:
1. Prompt Engineering
2. RAG (Retrieval-Augmented Generation)
3. Fine-tuning

## Comparison Metrics
- Exact Match Accuracy
- Precision, Recall, F1 per entity type
- Inference speed
- Resource requirements

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

from src.config import RESULTS_DIR
from src.evaluation import compare_methods

import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style("whitegrid")
%matplotlib inline

## 2. Load Results from All Methods

In [ ]:
# Load results
results_dict = {}

methods = {
    "Prompt Engineering": RESULTS_DIR / "prompt_engineering" / "Prompt Engineering_results.json",
    "RAG": RESULTS_DIR / "rag" / "RAG_results.json",
    "Fine-tuning": RESULTS_DIR / "finetuning" / "Fine-tuning_results.json",
}

for method_name, result_path in methods.items():
    if result_path.exists():
        with open(result_path, "r") as f:
            results_dict[method_name] = json.load(f)
        print(f"✓ Loaded {method_name} results")
    else:
        print(f"✗ {method_name} results not found at {result_path}")

print(f"\nLoaded {len(results_dict)} method results")

## 3. Create Comparison Table

In [ ]:
# Create comparison dataframe
comparison_df = compare_methods(results_dict)

# Add performance metrics
perf_data = []
for method_name, results in results_dict.items():
    perf_data.append({
        "Method": method_name,
        "Inference Time (s)": results.get("inference_time", 0),
        "Samples/Second": results.get("samples_per_second", 0),
    })

perf_df = pd.DataFrame(perf_data)
final_df = pd.merge(comparison_df, perf_df, on="Method")

# Display key columns
display_cols = [
    "Method",
    "Exact Match Accuracy",
    "macro_f1",
    "macro_precision",
    "macro_recall",
    "Inference Time (s)",
    "Samples/Second"
]

display_df = final_df[display_cols].copy()
display_df.columns = ["Method", "Exact Match", "F1", "Precision", "Recall", "Time (s)", "Samples/s"]

print("\n" + "="*100)
print("COMPARISON OF ALL METHODS")
print("="*100 + "\n")
print(display_df.to_string(index=False))

# Save to CSV
final_df.to_csv(RESULTS_DIR / "full_comparison.csv", index=False)
print(f"\nComparison saved to {RESULTS_DIR / 'full_comparison.csv'}")

## 4. Visualize Accuracy Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Exact Match Accuracy
axes[0].bar(display_df["Method"], display_df["Exact Match"], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_ylabel("Exact Match Accuracy")
axes[0].set_title("Exact Match Accuracy by Method")
axes[0].set_ylim(0, 1)
for i, v in enumerate(display_df["Exact Match"]):
    axes[0].text(i, v + 0.02, f"{v:.3f}", ha='center', va='bottom')

# Macro F1 Score
axes[1].bar(display_df["Method"], display_df["F1"], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[1].set_ylabel("Macro F1 Score")
axes[1].set_title("Macro F1 Score by Method")
axes[1].set_ylim(0, 1)
for i, v in enumerate(display_df["F1"]):
    axes[1].text(i, v + 0.02, f"{v:.3f}", ha='center', va='bottom')

plt.tight_layout()
plt.savefig(RESULTS_DIR / "accuracy_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

## 5. Per-Entity Type Comparison

In [ ]:
# Extract per-entity metrics
entity_types = ["person", "organizations", "address"]
metrics_data = []

for method_name, results in results_dict.items():
    partial_metrics = results["partial_match_metrics"]
    for entity_type in entity_types:
        if entity_type in partial_metrics:
            metrics = partial_metrics[entity_type]
            metrics_data.append({
                "Method": method_name,
                "Entity Type": entity_type,
                "Precision": metrics["precision"],
                "Recall": metrics["recall"],
                "F1": metrics["f1"]
            })

entity_df = pd.DataFrame(metrics_data)

# Plot F1 scores by entity type
fig, ax = plt.subplots(figsize=(12, 6))

entity_pivot = entity_df.pivot(index="Entity Type", columns="Method", values="F1")
entity_pivot.plot(kind="bar", ax=ax, width=0.8)

ax.set_ylabel("F1 Score")
ax.set_title("F1 Score by Entity Type and Method")
ax.set_ylim(0, 1)
ax.legend(title="Method")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.savefig(RESULTS_DIR / "entity_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print("\nPer-Entity F1 Scores:")
print(entity_pivot)

## 6. Inference Speed Comparison

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(display_df["Method"], display_df["Samples/s"], color=['#3498db', '#e74c3c', '#2ecc71'])
ax.set_xlabel("Samples per Second")
ax.set_title("Inference Speed Comparison")

for i, v in enumerate(display_df["Samples/s"]):
    ax.text(v + 0.01, i, f"{v:.2f}", va='center')

plt.tight_layout()
plt.savefig(RESULTS_DIR / "speed_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

## 7. Precision-Recall Analysis

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

colors = {'Prompt Engineering': '#3498db', 'RAG': '#e74c3c', 'Fine-tuning': '#2ecc71'}
markers = {'Prompt Engineering': 'o', 'RAG': 's', 'Fine-tuning': '^'}

for method_name in results_dict.keys():
    method_data = entity_df[entity_df["Method"] == method_name]
    
    ax.scatter(
        method_data["Recall"],
        method_data["Precision"],
        s=200,
        label=method_name,
        color=colors.get(method_name, 'gray'),
        marker=markers.get(method_name, 'o'),
        alpha=0.7
    )
    
    # Annotate entity types
    for _, row in method_data.iterrows():
        ax.annotate(
            row["Entity Type"][:3].upper(),
            (row["Recall"], row["Precision"]),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=8
        )

ax.set_xlabel("Recall")
ax.set_ylabel("Precision")
ax.set_title("Precision-Recall Comparison")
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1.05)
ax.set_ylim(0, 1.05)

# Add diagonal line for F1=0.5
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, linewidth=1)

plt.tight_layout()
plt.savefig(RESULTS_DIR / "precision_recall.png", dpi=300, bbox_inches='tight')
plt.show()

## 8. Summary and Recommendations

In [ ]:
print("\n" + "="*100)
print("SUMMARY AND RECOMMENDATIONS")
print("="*100 + "\n")

# Find best method for each metric
best_accuracy = display_df.loc[display_df["Exact Match"].idxmax()]
best_f1 = display_df.loc[display_df["F1"].idxmax()]
fastest = display_df.loc[display_df["Samples/s"].idxmax()]

print("🏆 Best Performance:")
print(f"  • Highest Exact Match Accuracy: {best_accuracy['Method']} ({best_accuracy['Exact Match']:.2%})")
print(f"  • Highest Macro F1: {best_f1['Method']} ({best_f1['F1']:.2%})")
print(f"  • Fastest Inference: {fastest['Method']} ({fastest['Samples/s']:.2f} samples/s)")

print("\n📊 Method Characteristics:\n")

print("1. Prompt Engineering:")
print("   ✓ Quick to implement (no training)")
print("   ✓ Good baseline performance")
print("   ✗ May be inconsistent with complex text")
print("   → Best for: Quick prototyping, simple use cases")

print("\n2. RAG (Retrieval-Augmented Generation):")
print("   ✓ Better accuracy with context")
print("   ✓ Reduced hallucination")
print("   ✗ Slower due to retrieval overhead")
print("   → Best for: Cases with good example corpus, domain adaptation")

print("\n3. Fine-tuning:")
print("   ✓ Highest accuracy for domain-specific patterns")
print("   ✓ Consistent predictions")
print("   ✗ Requires training time and GPU")
print("   → Best for: Production deployment, domain-specific requirements")

print("\n💡 Final Recommendation:")
if len(results_dict) >= 3:
    if best_f1['Method'] == 'Fine-tuning':
        print("   Fine-tuning shows the best overall performance and should be used")
        print("   for production deployment where accuracy is critical.")
    else:
        print(f"   {best_f1['Method']} shows the best F1 score and provides a good")
        print("   balance between performance and implementation complexity.")
else:
    print("   Run all three methods to get complete comparison.")

print("\n" + "="*100)

## 9. Export Results for Report

In [ ]:
# Create summary report
report = {
    "comparison_table": final_df.to_dict(orient="records"),
    "per_entity_metrics": entity_df.to_dict(orient="records"),
    "best_methods": {
        "exact_match": best_accuracy["Method"],
        "f1_score": best_f1["Method"],
        "fastest": fastest["Method"]
    },
    "detailed_results": results_dict
}

report_path = RESULTS_DIR / "final_report.json"
with open(report_path, "w") as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"\nFinal report saved to {report_path}")
print("\nAll comparison visualizations saved:")
print(f"  • {RESULTS_DIR / 'accuracy_comparison.png'}")
print(f"  • {RESULTS_DIR / 'entity_comparison.png'}")
print(f"  • {RESULTS_DIR / 'speed_comparison.png'}")
print(f"  • {RESULTS_DIR / 'precision_recall.png'}")